In [1]:
#Import the dependencies
import pandas as pd
import numpy as np
import datetime

# ORM dependencies
from sqlalchemy import create_engine, inspect, text

In [3]:
df = pd.read_csv("../Database/us_tornado_dataset_1950_2021.csv")
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67558 entries, 0 to 67557
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   yr      67558 non-null  int64  
 1   mo      67558 non-null  int64  
 2   dy      67558 non-null  int64  
 3   date    67558 non-null  object 
 4   st      67558 non-null  object 
 5   mag     67558 non-null  int64  
 6   inj     67558 non-null  int64  
 7   fat     67558 non-null  int64  
 8   slat    67558 non-null  float64
 9   slon    67558 non-null  float64
 10  elat    67558 non-null  float64
 11  elon    67558 non-null  float64
 12  len     67558 non-null  float64
 13  wid     67558 non-null  int64  
dtypes: float64(5), int64(7), object(2)
memory usage: 7.2+ MB


In [5]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67558 entries, 0 to 67557
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   yr      67558 non-null  int64         
 1   mo      67558 non-null  int64         
 2   dy      67558 non-null  int64         
 3   date    67558 non-null  datetime64[ns]
 4   st      67558 non-null  object        
 5   mag     67558 non-null  int64         
 6   inj     67558 non-null  int64         
 7   fat     67558 non-null  int64         
 8   slat    67558 non-null  float64       
 9   slon    67558 non-null  float64       
 10  elat    67558 non-null  float64       
 11  elon    67558 non-null  float64       
 12  len     67558 non-null  float64       
 13  wid     67558 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(7), object(1)
memory usage: 7.2+ MB


In [7]:
renamed_df = df.rename(columns={
                                'yr': 'year', 
                                'mo': 'month', 
                                'dy': 'day', 
                                'date': 'date', 
                                'st': 'state', 
                                'mag': 'tornado_magnitude', 
                                'inj': 'injuries', 
                                'fat': 'fatalities', 
                                'slat': 'start_latitude', 
                                'slon': 'start_longitude', 
                                'elat': 'end_latitude', 
                                'elon': 'end_longitude', 
                                'len': 'tornado_length', 
                                'wid': 'tornado_width'
                                })

renamed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67558 entries, 0 to 67557
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   year               67558 non-null  int64         
 1   month              67558 non-null  int64         
 2   day                67558 non-null  int64         
 3   date               67558 non-null  datetime64[ns]
 4   state              67558 non-null  object        
 5   tornado_magnitude  67558 non-null  int64         
 6   injuries           67558 non-null  int64         
 7   fatalities         67558 non-null  int64         
 8   start_latitude     67558 non-null  float64       
 9   start_longitude    67558 non-null  float64       
 10  end_latitude       67558 non-null  float64       
 11  end_longitude      67558 non-null  float64       
 12  tornado_length     67558 non-null  float64       
 13  tornado_width      67558 non-null  int64         
dtypes: dat

In [9]:
renamed_df.head()

,year,month,day,date,state,tornado_magnitude,injuries,fatalities,start_latitude,start_longitude,end_latitude,end_longitude,tornado_length,tornado_width
0,1950,1,3,1950-01-03,IL,3,3,0,39.10,-89.30,39.12,-89.23,3.6,130
1,1950,1,3,1950-01-03,MO,3,3,0,38.77,-90.22,38.83,-90.03,9.5,150
2,1950,1,3,1950-01-03,OH,1,1,0,40.88,-84.58,0.00,0.00,0.1,10
3,1950,1,13,1950-01-13,AR,3,1,1,34.40,-94.37,0.00,0.00,0.6,17
4,1950,1,25,1950-01-25,IL,2,0,0,41.17,-87.33,0.00,0.00,0.1,100


In [11]:
# Find duplicate rows but keep the first occurrence
duplicates_to_remove = renamed_df[renamed_df.duplicated(keep='first')]

# Display the duplicates that will be removed
duplicates_to_remove.shape
duplicates_to_remove.info()


<class 'pandas.core.frame.DataFrame'>
Index: 462 entries, 391 to 64676
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   year               462 non-null    int64         
 1   month              462 non-null    int64         
 2   day                462 non-null    int64         
 3   date               462 non-null    datetime64[ns]
 4   state              462 non-null    object        
 5   tornado_magnitude  462 non-null    int64         
 6   injuries           462 non-null    int64         
 7   fatalities         462 non-null    int64         
 8   start_latitude     462 non-null    float64       
 9   start_longitude    462 non-null    float64       
 10  end_latitude       462 non-null    float64       
 11  end_longitude      462 non-null    float64       
 12  tornado_length     462 non-null    float64       
 13  tornado_width      462 non-null    int64         
dtypes: datetime

In [13]:
df_cleaned = renamed_df.drop_duplicates(keep='first')

# Display the cleaned DataFrame
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67096 entries, 0 to 67557
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   year               67096 non-null  int64         
 1   month              67096 non-null  int64         
 2   day                67096 non-null  int64         
 3   date               67096 non-null  datetime64[ns]
 4   state              67096 non-null  object        
 5   tornado_magnitude  67096 non-null  int64         
 6   injuries           67096 non-null  int64         
 7   fatalities         67096 non-null  int64         
 8   start_latitude     67096 non-null  float64       
 9   start_longitude    67096 non-null  float64       
 10  end_latitude       67096 non-null  float64       
 11  end_longitude      67096 non-null  float64       
 12  tornado_length     67096 non-null  float64       
 13  tornado_width      67096 non-null  int64         
dtypes: datetime

In [15]:
# Write to the database
# Create the connection engine
engine = create_engine("sqlite:///../Database/us_tornado.sqlite")

# Define the batch size
batch_size = 1000 

# Write to SQL (USING con=engine)
df_cleaned.to_sql(name="us_tornado", con=engine, index=False, if_exists="append", method="multi", chunksize=batch_size)

67096

In [17]:
# Open the Threads
conn = engine.connect() # Raw SQL/Pandas

In [19]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within the 'dow' table and its types
for table in tables:
    print(table)
    print("--------")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

us_tornado
--------
year BIGINT
month BIGINT
day BIGINT
date DATETIME
state TEXT
tornado_magnitude BIGINT
injuries BIGINT
fatalities BIGINT
start_latitude FLOAT
start_longitude FLOAT
end_latitude FLOAT
end_longitude FLOAT
tornado_length FLOAT
tornado_width BIGINT



In [21]:
engine.dispose()